## Trying out a linear model: 

There have been a few [great](https://www.kaggle.com/comartel/house-prices-advanced-regression-techniques/house-price-xgboost-starter/run/348739)  [scripts](https://www.kaggle.com/zoupet/house-prices-advanced-regression-techniques/xgboost-10-kfolds-with-scikit-learn/run/357561) on [xgboost](https://www.kaggle.com/tadepalli/house-prices-advanced-regression-techniques/xgboost-with-n-trees-autostop-0-12638/run/353049) already so I'd figured I'd try something simpler: a regularized linear regression model. Surprisingly it does really well with very little feature engineering. The key point is to to log_transform the numeric variables since most of them are skewed.

In [104]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import skew

%config InlineBackend.figure_format = 'png'

In [105]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")


In [106]:
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))


In [107]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)
prices = pd.DataFrame({"price":train["SalePrice"], "log(price + 1)":np.log1p(train["SalePrice"])})

In [108]:
#log transform the target:
train["SalePrice"] = np.log1p(train["SalePrice"])

#log transform skewed numeric features:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

In [109]:
all_data = pd.get_dummies(all_data)
print (all_data)

      MSSubClass  LotFrontage    LotArea  OverallQual  OverallCond  YearBuilt  \
0       4.110874     4.189655   9.042040            7            5       2003   
1       3.044522     4.394449   9.169623            6            8       1976   
2       4.110874     4.234107   9.328212            7            5       2001   
3       4.262680     4.110874   9.164401            7            5       1915   
4       4.110874     4.442651   9.565284            8            5       2000   
5       3.931826     4.454347   9.555064            5            5       1993   
6       3.044522     4.330733   9.218804            8            5       2004   
7       4.110874          NaN   9.247925            7            6       1973   
8       3.931826     3.951244   8.719481            7            5       1931   
9       5.252273     3.931826   8.912069            5            6       1939   
10      3.044522     4.262680   9.323758            5            5       1965   
11      4.110874     4.45434

In [110]:
#filling NA's with the mean of the column:
all_data = all_data.fillna(all_data[:train.shape[0]].mean())

In [111]:
#creating matrices for sklearn:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.SalePrice

In [112]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

In [113]:
model_lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005]).fit(X_train, y)

In [114]:
preds = np.expm1(model_lasso.predict(X_test))

In [115]:
solution = pd.DataFrame({"id":test.Id, "SalePrice":preds})
solution.to_csv("new_ridge_sol.csv", index = False)